<a href="https://colab.research.google.com/github/Sathyakumarnsk/action_classifier/blob/master/lstm_N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
%tensorflow_version 1.x
import json
import numpy as np
#from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Input
from keras.layers.normalization import BatchNormalization
from tensorflow.keras import regularizers
from keras.optimizers import Adam, SGD
from keras.layers import LeakyReLU

In [4]:
#Loading in data
data_path = "./data_humans.json"
#Load data
with open(data_path, "r") as fp:
    data = json.load(fp)


In [7]:
# data.keys()
labels=data['metadata']['all labels in loaded order']
# data['metadata'].keys()
#data['dataperclass'].keys()

In [8]:
X=list(); Y=list()
C_x=list();C_y=list()
for i,c in enumerate(labels):
  print(c)
  skel_class=data['dataperclass'][c]['skelsinfo_list']
  #print(np.array(skel_class))
  C_x.append(skel_class)
  print(np.array(C_x).shape)
  label_class=data['dataperclass'][c]['label_list']
  C_y.append(label_class)
  #print(len(C_y))
  X.extend(skel_class)
  Y.extend(label_class)

follow
(1, 500, 24)
pour
(2, 500, 24)
pp
(3, 500, 24)
stand
(4, 500, 24)
stop
(5, 500, 24)
wave
(6, 500, 24)


In [9]:
print(np.array(X).shape)
print(np.array(Y).shape)

(3000, 24)
(3000,)


In [10]:
targets=[]
for y in Y:
  targets.append(y-1)

In [11]:
encoder_Y = to_categorical(targets)
print(Y[0], ": ", encoder_Y[0])
print(Y[50], ": ", encoder_Y[50])
print(Y[100], ": ", encoder_Y[100])
print(Y[150], ": ", encoder_Y[150])
print(Y[1000], ": ", encoder_Y[1000])

1 :  [1. 0. 0. 0. 0. 0.]
1 :  [1. 0. 0. 0. 0. 0.]
1 :  [1. 0. 0. 0. 0. 0.]
1 :  [1. 0. 0. 0. 0. 0.]
3 :  [0. 0. 1. 0. 0. 0.]


In [13]:
import numpy as np
def train_test_split(X_data,Y_data,val=0.2,test=0.1):
    X_data=np.array(X_data);Y_data=np.array(Y_data)
    # create random train/test split
    indices = list(range(X_data.shape[0]))
    train_split=1.0-(val+test)
    num_training_indices = int(train_split * X_data.shape[0])
    num_valid_indices = int(val * X_data.shape[0])
    #num_test_indices = int(train_split * X_data.shape[0])
    np.random.shuffle(indices)
    train_indices = indices[:num_training_indices]
    valid_indices=indices[num_training_indices:num_training_indices+num_valid_indices]
    test_indices = indices[num_training_indices+num_valid_indices:]

    # split the actual data
    X_data_train, X_data_val, X_data_test = X_data[train_indices], X_data[valid_indices], X_data[test_indices]
    Y_data_train, Y_data_val ,Y_data_test = Y_data[train_indices], Y_data[valid_indices], Y_data[test_indices]
    return (X_data_train, X_data_val, X_data_test, Y_data_train, Y_data_val, Y_data_test)


In [ ]:
from tensorflow.keras.utils import normalize
#from keras.preprocessing import timeseries_dataset_from_array
X=normalize(X)

In [61]:

def batcher(l,y,n=10):
  batches=[]
  labels_b=[]
  for i in range(0,len(l),1):
    if (i<len(l)-n):
      batches.append(l[i:n+i])
      labels_b.append(y[i:n+i])
  return batches,labels_b
      #print(l[i:n+i])

batches,labels_batches=batcher(X,encoder_Y) 

X_train, X_val, X_test, Y_train, Y_val, Y_test=train_test_split(batches,labels_batches)

In [75]:
X_train.shape


(2093, 10, 24)

In [65]:
def build_model(input_shape):
    """Generates RNN-LSTM model
    :param input_shape (tuple): Shape of input set
    :return model: RNN-LSTM model
    """

    # build network topology
    model = keras.Sequential()

    # 2 LSTM layers
    model.add(keras.layers.LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.LSTM(64))

    # dense layer
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    # output layer
    model.add(keras.layers.Dense(10, activation='softmax'))

    return model


In [72]:
model=build_model((10,24))
model.summary()
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 10, 64)            22784     
_________________________________________________________________
lstm_9 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                650       
Total params: 60,618
Trainable params: 60,618
Non-trainable params: 0
_________________________________________________________________


In [73]:
# batch_size: number of samples per gradient update
# epochs: how many times to pass through the whole training set
# verbose: show one line for every completed epoch
history=model.fit(X_train, Y_train, batch_size=24, epochs=100, verbose=2, validation_data=(X_val, Y_val))
#model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2093 samples, validate on 598 samples
Epoch 1/100


InvalidArgumentError: ignored